In [8]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [9]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [10]:
TRAINING_DIR = "C:/Users/rifat/OneDrive/Desktop/Dataset/Train"
train_datagen = ImageDataGenerator(rescale=1.0/255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=10, target_size=(150, 150))
VALIDATION_DIR = "C:/Users/rifat/OneDrive/Desktop/Dataset/Test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, batch_size=10, target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [11]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.h5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history = model.fit(train_generator,epochs=20,validation_data=validation_generator,callbacks=[checkpoint])

Epoch 1/20
132/132 [==============================] - 98s 732ms/step - loss: 0.7145 - acc: 0.5202 - val_loss: 0.4310 - val_acc: 0.8660
Epoch 2/20
132/132 [==============================] - 93s 704ms/step - loss: 0.4125 - acc: 0.8228 - val_loss: 0.1432 - val_acc: 0.9485
Epoch 3/20
132/132 [==============================] - 96s 728ms/step - loss: 0.2874 - acc: 0.8867 - val_loss: 0.1078 - val_acc: 0.9639
Epoch 4/20
132/132 [==============================] - 103s 777ms/step - loss: 0.3013 - acc: 0.8913 - val_loss: 0.1667 - val_acc: 0.9124
Epoch 5/20
132/132 [==============================] - 78s 589ms/step - loss: 0.2511 - acc: 0.8943 - val_loss: 0.1326 - val_acc: 0.9536
Epoch 6/20
132/132 [==============================] - 76s 573ms/step - loss: 0.2079 - acc: 0.9232 - val_loss: 0.0491 - val_acc: 0.9794
Epoch 7/20
132/132 [==============================] - 95s 719ms/step - loss: 0.2041 - acc: 0.9179 - val_loss: 0.0382 - val_acc: 0.9845
Epoch 8/20
132/132 [==============================] - 

In [1]:
import cv2
from keras.models import load_model
import numpy as np
model=load_model("./model2-016.h5")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('C:/Users/rifat/AppData/Local/Programs/Python/Python311/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 37ms/step
